In [14]:
#mask filling
import torch
import numpy as np

tgt_length = 10
mask_begin = 9
tgt_mask = torch.zeros(size = (tgt_length,), dtype = int)
tgt_mask[:mask_begin] = 1

mask_mat = torch.zeros(size  = (tgt_length, tgt_length), 
                       dtype = torch.float)

mask_mat_part = torch.triu(
    torch.full(
        size       = (tgt_length - mask_begin - 1, 
                      tgt_length - mask_begin - 1), 
        fill_value = -torch.inf,
        dtype      = float)
    )

mask_mat[:tgt_length - mask_begin - 1, mask_begin + 1:] = mask_mat_part
mask_mat
#mask_mat_part.shape


tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [10]:
import torch

tgt_length = 10
mask_begin = 9
n_batch = 3 
mask = torch.zeros(size = (n_batch, tgt_length), dtype = int)

for i in range(n_batch):
    mask[i, :(2 * i + 3) % tgt_length] = 1


mask_starts = [len(mask[i,:][mask[i,:] == 1]) for i in range(n_batch)]
mask

tensor([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0]])

In [32]:
src_mask = torch.einsum('bl,bL->blL', mask, mask)
#src_mask = ~src_mask.to(dtype = torch.bool)
src_mask = (src_mask == 0)
mask_mat[src_mask] = -torch.inf

mask_mat[0]

tensor([[0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf]])

In [ ]:
import torch

a = torch.tensor([[1,1,1,1],
                  [2,2,2,2],
                  [3,3,3,3]])

b = torch.tensor([[4,4,4,4],
                  [5,5,5,5],
                  [6,6,6,6]])

c = torch.tensor([[7,7,7,7],
                  [8,8,8,8],
                  [9,9,9,9]])

cat = torch.cat(tensors=[a,b,c], dim = 0)
cat.shape

torch.Size([9, 4])

In [32]:
import torch
import math

d_model = 16
seq_len = 10

sin = math.sin
cos = math.cos
wave = [torch.tensor([10000 ** (i / d_model), 10000**((i + 1) / d_model)]) for i in range(0, d_model, 2)]
wave = torch.concat(wave, dim = 0)
Iwave = torch.ones_like(wave)

pos = torch.arange(0, seq_len, 1)
Ipos = torch.ones_like(pos)

onePE = torch.einsum('i,j->ij', pos, Iwave) / torch.einsum('i,j->ij', Ipos, wave)
onePE[:,0::2] = torch.sin(onePE[:,0::2])
onePE[:,1::2] = torch.cos(onePE[:,1::2])

PE = [onePE.tolist() for _ in range(4)]
torch.tensor(PE)

tensor([[[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
           1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,
           0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
           1.0000e+00],
         [ 8.4147e-01,  8.4601e-01,  3.1098e-01,  9.8423e-01,  9.9833e-02,
           9.9842e-01,  3.1618e-02,  9.9984e-01,  9.9998e-03,  9.9998e-01,
           3.1623e-03,  1.0000e+00,  1.0000e-03,  1.0000e+00,  3.1623e-04,
           1.0000e+00],
         [ 9.0930e-01,  4.3146e-01,  5.9113e-01,  9.3742e-01,  1.9867e-01,
           9.9368e-01,  6.3203e-02,  9.9937e-01,  1.9999e-02,  9.9994e-01,
           6.3245e-03,  9.9999e-01,  2.0000e-03,  1.0000e+00,  6.3246e-04,
           1.0000e+00],
         [ 1.4112e-01, -1.1597e-01,  8.1265e-01,  8.6104e-01,  2.9552e-01,
           9.8580e-01,  9.4726e-02,  9.9858e-01,  2.9995e-02,  9.9986e-01,
           9.4867e-03,  9.9999e-01,  3.0000e-03,  1.0000e+00,  9.4868e-04,
           1.0000e+00],
    